In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to the 'fer2013' dataset directory
train_dir = 'fer2013plus/fer2013/train'
test_dir = 'fer2013plus/fer2013/test'

# Create an ImageDataGenerator for data augmentation
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,  # Random rotation
    width_shift_range=0.1,  # Random horizontal shift
    height_shift_range=0.1,  # Random vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
)

# Create an ImageDataGenerator for the test data (only rescale, no augmentation)
datagen_test = ImageDataGenerator(rescale=1./255)

# Create empty lists to store image paths and labels
train_image_paths = []
train_labels = []
test_image_paths = []
test_labels = []

# Function to load image paths and labels
def load_image_paths_and_labels(directory, class_label):
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            filepath = os.path.join(directory, filename)
            train_image_paths.append(filepath)
            train_labels.append(class_label)

# Load image paths and labels for each class in the train directory
load_image_paths_and_labels(os.path.join(train_dir, 'happiness'), 0)
load_image_paths_and_labels(os.path.join(train_dir, 'disgust'), 1)
load_image_paths_and_labels(os.path.join(train_dir, 'surprise'), 2)
load_image_paths_and_labels(os.path.join(train_dir, 'fear'), 3)
load_image_paths_and_labels(os.path.join(train_dir, 'contempt'), 4)
load_image_paths_and_labels(os.path.join(train_dir, 'neutral'), 5)
load_image_paths_and_labels(os.path.join(train_dir, 'anger'), 6)
load_image_paths_and_labels(os.path.join(train_dir, 'sadness'), 7)

# Convert the lists to NumPy arrays
train_image_paths = np.array(train_image_paths)
train_labels = np.array(train_labels)

# Perform data augmentation on the training data and create a balanced dataset
num_augmented_samples_per_class = 1000
augmented_data = []
train=datagen_train.flow_from_directory(
        train_dir,
        target_size=(48, 48),
        batch_size=1,
        class_mode='sparse',
        color_mode='grayscale',
        shuffle=True,
    )
for _ in range(num_augmented_samples_per_class):
    for batch in train:
        if len(augmented_data) >= num_augmented_samples_per_class * len(np.unique(train_labels)):
            break
        augmented_data.append(batch[0][0])
        train_labels = np.append(train_labels, batch[0][1])

# Convert the augmented data and labels back to lists
augmented_data = np.array(augmented_data)
train_labels = train_labels.astype(int).tolist()

# Load image paths and labels for the test directory
load_image_paths_and_labels(os.path.join(test_dir, 'happiness'), 0)
load_image_paths_and_labels(os.path.join(test_dir, 'disgust'), 1)
load_image_paths_and_labels(os.path.join(test_dir, 'surprise'), 2)
load_image_paths_and_labels(os.path.join(test_dir, 'fear'), 3)
load_image_paths_and_labels(os.path.join(test_dir, 'contempt'), 4)
load_image_paths_and_labels(os.path.join(test_dir, 'neutral'), 5)
load_image_paths_and_labels(os.path.join(test_dir, 'anger'), 6)
load_image_paths_and_labels(os.path.join(test_dir, 'sadness'), 7)

# Convert the lists to NumPy arrays
test_image_paths = np.array(test_image_paths)
test_labels = np.array(test_labels)

# Create TensorFlow datasets for training and testing
batch_size = 64

train_dataset = tf.data.Dataset.from_tensor_slices((augmented_data, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=len(augmented_data)).batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_image_paths, test_labels))
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)


2023-08-03 17:50:51.966522: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-03 17:50:51.992076: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-08-03 17:50:51.992109: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-08-03 17:50:51.992126: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-03 17:50:51.997135: I tensorflow/core/platform/cpu_feature_g

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


2023-08-03 17:50:52.462145: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 28386 images belonging to 8 classes.


2023-08-03 17:50:53.899408: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1885] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1200 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


InvalidArgumentError: {{function_node __wrapped__ShuffleDatasetV3_device_/job:localhost/replica:0/task:0/device:CPU:0}} buffer_size must be greater than zero or UNKNOWN_CARDINALITY [Op:ShuffleDatasetV3] name: 